In [1]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

import numpy as np
import json

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from google.cloud import bigquery
from witwidget.notebook.visualization import WitWidget, WitConfigBuilder

In [2]:
query="""
SELECT
  weight_pounds,
  is_male,
  mother_age,
  plurality,
  gestation_weeks
FROM
  publicdata.samples.natality
WHERE year > 2000
LIMIT 10000
"""
df = bigquery.Client().query(query).to_dataframe()
df.head()

,weight_pounds,is_male,mother_age,plurality,gestation_weeks
0,7.109908,False,25,1,38.0
1,7.588311,False,19,1,40.0
2,4.812691,True,35,1,33.0
3,7.251004,True,30,2,38.0
4,6.206013,False,21,1,36.0


In [3]:
df.describe()

,weight_pounds,mother_age,plurality,gestation_weeks
count,9994.000000,10000.000000,10000.000000,9873.000000
mean,7.267343,27.299200,1.035300,38.662919
std,1.345904,6.190841,0.197124,2.560298
min,0.595248,13.000000,1.000000,19.000000
25%,6.563162,22.000000,1.000000,38.000000
50%,7.373360,27.000000,1.000000,39.000000
75%,8.124034,32.000000,1.000000,40.000000
max,13.000660,54.000000,4.000000,47.000000


In [4]:
df['is_male'].value_counts()

True     5097
False    4903
Name: is_male, dtype: int64

In [5]:
df = df.dropna()
df = shuffle(df, random_state=2)

In [6]:
labels = df['weight_pounds']
data = df.drop(columns=['weight_pounds'])
data['is_male'] = data['is_male'].astype(int)

In [7]:
data.head()

,is_male,mother_age,plurality,gestation_weeks
3326,1,27,1,39.0
3717,0,29,1,38.0
8767,0,23,1,38.0
3469,0,24,1,36.0
501,0,21,1,40.0


In [8]:
x,y = data,labels
x_train,x_test,y_train,y_test = train_test_split(x,y)

In [9]:
model = Sequential([
    Dense(64, activation='relu', input_shape=(len(x_train.iloc[0]),)),
    Dense(32, activation='relu'),
    Dense(1)]
)

In [10]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(),
              loss=tf.keras.losses.MeanSquaredError(),
              metrics=['mae', 'mse'])

In [11]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 64)                320       
_________________________________________________________________
dense_1 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 33        
Total params: 2,433
Trainable params: 2,433
Non-trainable params: 0
_________________________________________________________________


In [12]:
model.fit(x_train, y_train, epochs=10, validation_split=0.1)

Train on 6661 samples, validate on 741 samples
Epoch 1/10
6661/6661 [==============================] - 2s 240us/sample - loss: 1.7876 - mae: 0.9973 - mse: 1.7876 - val_loss: 2.0861 - val_mae: 1.1527 - val_mse: 2.0861
Epoch 2/10
6661/6661 [==============================] - 1s 144us/sample - loss: 1.4802 - mae: 0.9445 - mse: 1.4802 - val_loss: 1.5774 - val_mae: 0.9923 - val_mse: 1.5774
Epoch 3/10
6661/6661 [==============================] - 1s 138us/sample - loss: 1.4145 - mae: 0.9255 - mse: 1.4145 - val_loss: 1.3906 - val_mae: 0.9203 - val_mse: 1.3906
Epoch 4/10
6661/6661 [==============================] - 1s 156us/sample - loss: 1.4025 - mae: 0.9187 - mse: 1.4025 - val_loss: 1.4202 - val_mae: 0.9268 - val_mse: 1.4202
Epoch 5/10
6661/6661 [==============================] - 1s 129us/sample - loss: 1.3749 - mae: 0.9127 - mse: 1.3749 - val_loss: 1.3432 - val_mae: 0.8963 - val_mse: 1.3432
Epoch 6/10
6661/6661 [==============================] - 1s 179us/sample - loss: 1.3512 - mae: 0.9055 - 

In [14]:
num_examples = 10
predictions = model.predict(x_test[:num_examples])

In [15]:
for i in range(num_examples):
    print('Predicted val: ', predictions[i][0])
    print('Actual val: ',y_test.iloc[i])
    print()

Predicted val:  7.2519293
Actual val:  7.31273323054

Predicted val:  7.71146
Actual val:  6.4926136159

Predicted val:  7.6620502
Actual val:  7.42075973892

Predicted val:  7.115779
Actual val:  7.7492485093

Predicted val:  7.265236
Actual val:  6.30962993844

Predicted val:  7.406226
Actual val:  8.87581066812

Predicted val:  6.882679
Actual val:  7.62578964258

Predicted val:  7.194585
Actual val:  8.062304921339999

Predicted val:  7.4086666
Actual val:  8.000575487979999

Predicted val:  7.6238985
Actual val:  9.25059651352

